# Mine Goals and Assists for Currently Active Players
We will need to mine current NHL rosters using the API. We will reuse filtered even strength data from previous work

In [118]:
import numpy as np 
import pandas as pd 
import pickle    
import requests
import pprint

In [29]:
# API header
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', 
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate"
}

In [2]:
# load dataset
with open('input/2019fiveVfive.pkl', 'rb') as f:
    game_data = pickle.load(f)

### Get Roster Info
Start by mining team IDs

In [42]:
# get a list of team IDs
team_ids = []
r = requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
d = r.json()

for team in d['teams']:
    team_ids.append(team['id'])
print(len(team_ids))


31


Loop through team IDs and mine each teams roster (fullName)

In [51]:
stats = {}

for team_id in team_ids:
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/teams/' + str(team_id) + '/roster')
    d = r.json()
    for player in d['roster']:
        name = player['person']['fullName']
        stats[name] = {'goals' : 0, 'assists' : 0}

print(len(stats))        

1153


### Get even-strength stats from 2019-20 season (Goals and Assists)

In [67]:
# reset stats to zeros
for player in stats:
    stats[player]['goals'] = 0
    stats[player]['assists'] = 0
    
# start counting from first game of season
for i in range(1, len(game_data)): 
    if game_data[i]['result']['event'] == 'Goal':
        #print(game_data[i])
        for player in game_data[i]['players']:
            if player['playerType'] == 'Scorer':        # Increment player's goal count
                name = player['player']['fullName']
                if name in stats:
                    stats[name]['goals'] = stats[name]['goals'] + 1
            if player['playerType'] == 'Assist':        # Increment player's assist count
                name = player['player']['fullName']
                if name in stats:
                    stats[name]['assists'] = stats[name]['assists'] + 1
print(stats)

{'Travis Zajac': {'goals': 7, 'assists': 14}, 'P.K. Subban': {'goals': 5, 'assists': 7}, 'Kyle Palmieri': {'goals': 14, 'assists': 13}, 'Dmitry Kulikov': {'goals': 2, 'assists': 8}, 'Sami Vatanen': {'goals': 4, 'assists': 7}, 'Scott Wedgewood': {'goals': 0, 'assists': 0}, 'Ben Street': {'goals': 0, 'assists': 1}, 'Matt Tennyson': {'goals': 0, 'assists': 3}, 'Ryan Murray': {'goals': 2, 'assists': 5}, 'Damon Severson': {'goals': 6, 'assists': 16}, 'Connor Carrick': {'goals': 1, 'assists': 5}, 'Nikita Gusev': {'goals': 14, 'assists': 21}, 'Andreas Johnsson': {'goals': 5, 'assists': 12}, 'Will Butcher': {'goals': 3, 'assists': 15}, 'Miles Wood': {'goals': 11, 'assists': 12}, 'Josh Jacobs': {'goals': 0, 'assists': 0}, 'Pavel Zacha': {'goals': 3, 'assists': 15}, 'Mackenzie Blackwood': {'goals': 0, 'assists': 0}, 'Nicholas Merkley': {'goals': 1, 'assists': 1}, 'Colton White': {'goals': 0, 'assists': 0}, 'Brett Seney': {'goals': 0, 'assists': 0}, 'Brandon Gignac': {'goals': 0, 'assists': 0}, '

### Check totals

In [73]:
goals = 0
assists = 0
for player in stats:
    goals = goals + stats[player]['goals']
    assists = assists + stats[player]['assists']

print('Goals: ' + str(goals))
print('Assists: ' + str(assists))
print('Total Points: ' + str(goals + assists))

Goals: 4830
Assists: 7592
Total Points: 12422


### Store data with pickle

In [76]:
year = '2019'
with open('./input/'+year+'evStats.pkl', 'wb') as f:
    pickle.dump(game_data, f, pickle.HIGHEST_PROTOCOL)

# Calculate Even Strength Totals for Lines
We will use DailyFaceoff.com for current line combinations

In [120]:
#### Via DailyFaceoff.com ####
line_players = {
    'CAR_F1' : {'Sebastian Aho', 'Teuvo Teravainen', 'Nino Niederreiter'},
    'CAR_F2' : {'Jesper Fast', 'Vincent Trocheck', 'Andrei Svechnikov'},
    'CAR_F3' : {'Warren Foegele', 'Jordan Staal', 'Martin Necas'},
    'CAR_F4' : {'Ryan Dzingel', 'Morgan Geekie', 'Brock McGinn'},
    
    'EDM_F1' : {'Ryan Nugent-Hopkins', 'Connor McDavid', 'Zack Kassian'},
    'EDM_F2' : {'Dominik Kahun', 'Leon Draisaitl', 'Kailer Yamamoto'},
    'EDM_F3' : {'Tyler Ennis', 'Kyle Turris', 'Jesse Puljujarvi'},
    'EDM_F4' : {'Josh Archibald', 'Jujhar Khaira', 'Alex Chiasson'},
    
    'TOR_F1' : {'Joe Thornton', 'Auston Matthews', 'Mitchell Marner'},
    'TOR_F2' : {'Jimmy Vesey', 'John Tavares', 'William Nylander'},
    'TOR_F3' : {'Ilya Mikheyev', 'Alexander Kerfoot', 'Zach Hyman'},
    'TOR_F4' : {'Alexander Barabanov', 'Jason Spezza', 'Wayne Simmonds'},
}

line_totals = {
    'CAR_F1' : {'goals': 0, 'assists' : 0},
    'CAR_F2' : {'goals': 0, 'assists' : 0},
    'CAR_F3' : {'goals': 0, 'assists' : 0},
    'CAR_F4' : {'goals': 0, 'assists' : 0},
    
    'EDM_F1' : {'goals': 0, 'assists' : 0},
    'EDM_F2' : {'goals': 0, 'assists' : 0},
    'EDM_F3' : {'goals': 0, 'assists' : 0},
    'EDM_F4' : {'goals': 0, 'assists' : 0},
    
    'TOR_F1' : {'goals': 0, 'assists' : 0},
    'TOR_F2' : {'goals': 0, 'assists' : 0},
    'TOR_F3' : {'goals': 0, 'assists' : 0},
    'TOR_F4' : {'goals': 0, 'assists' : 0},
}

In [121]:
pprint.pprint(line_totals)

{'CAR_F1': {'assists': 0, 'goals': 0},
 'CAR_F2': {'assists': 0, 'goals': 0},
 'CAR_F3': {'assists': 0, 'goals': 0},
 'CAR_F4': {'assists': 0, 'goals': 0},
 'EDM_F1': {'assists': 0, 'goals': 0},
 'EDM_F2': {'assists': 0, 'goals': 0},
 'EDM_F3': {'assists': 0, 'goals': 0},
 'EDM_F4': {'assists': 0, 'goals': 0},
 'TOR_F1': {'assists': 0, 'goals': 0},
 'TOR_F2': {'assists': 0, 'goals': 0},
 'TOR_F3': {'assists': 0, 'goals': 0},
 'TOR_F4': {'assists': 0, 'goals': 0}}


In [122]:
# reset line_totals to zeros
for line in line_totals:
    line_totals[line]['goals'] = 0
    line_totals[line]['assists'] = 0

# loop through all players, if player is in one of the lines, append their goals/assists to line_totals
for player in stats:
    for line in line_players:
        if player in line_players[line]:
            print(line + ' ' + player + ' G' + str(stats[player]['goals']) + ' A' + str(stats[player]['assists']))
            line_totals[line]['goals'] = line_totals[line]['goals'] + stats[player]['goals']
            line_totals[line]['assists'] = line_totals[line]['assists'] + stats[player]['assists']
pprint.pprint(line_totals)

CAR_F1 Sebastian Aho G26 A18
TOR_F1 Joe Thornton G4 A19
TOR_F4 Jason Spezza G9 A11
TOR_F4 Wayne Simmonds G3 A13
TOR_F2 John Tavares G18 A19
TOR_F3 Zach Hyman G17 A13
TOR_F2 Jimmy Vesey G8 A10
TOR_F3 Alexander Kerfoot G8 A17
TOR_F2 William Nylander G23 A20
TOR_F1 Mitchell Marner G8 A32
TOR_F1 Auston Matthews G35 A19
TOR_F3 Ilya Mikheyev G7 A15
TOR_F4 Alexander Barabanov G0 A0
CAR_F3 Jordan Staal G6 A17
CAR_F1 Nino Niederreiter G7 A10
CAR_F2 Jesper Fast G10 A17
CAR_F4 Ryan Dzingel G4 A16
CAR_F2 Vincent Trocheck G12 A21
CAR_F1 Teuvo Teravainen G12 A28
CAR_F4 Brock McGinn G5 A10
CAR_F3 Warren Foegele G10 A17
CAR_F4 Morgan Geekie G3 A1
CAR_F3 Martin Necas G11 A14
CAR_F2 Andrei Svechnikov G20 A22
EDM_F3 Kyle Turris G7 A14
EDM_F3 Tyler Ennis G12 A16
EDM_F4 Alex Chiasson G5 A6
EDM_F1 Zack Kassian G15 A19
EDM_F4 Josh Archibald G9 A7
EDM_F1 Ryan Nugent-Hopkins G15 A21
EDM_F4 Jujhar Khaira G6 A4
EDM_F2 Leon Draisaitl G28 A38
EDM_F1 Connor McDavid G23 A31
EDM_F3 Jesse Puljujarvi G0 A0
EDM_F2 Kaile

In [123]:
pd.DataFrame.from_dict(line_totals)

,CAR_F1,CAR_F2,CAR_F3,CAR_F4,EDM_F1,EDM_F2,EDM_F3,EDM_F4,TOR_F1,TOR_F2,TOR_F3,TOR_F4
goals,45,42,27,12,53,48,19,20,47,49,32,12
assists,56,60,48,27,71,71,30,17,70,49,45,24
